<a href="https://colab.research.google.com/github/boffett/paytm_test/blob/main/PaytmTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install pyspark

In [37]:
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, DateType, FloatType
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

In [15]:
path = "/content/paytm_test/"
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [68]:
movie_schema = StructType([
    StructField("MovieID", IntegerType(), True),
    StructField("Title", StringType(), True),
    StructField("Genres", StringType(), True)
])

movie_df = spark.read.csv(path + "movies.dat", sep="::", header=False,
        schema=movie_schema, mode='DROPMALFORMED')
movie_df = movie_df.dropna()
movie_df.cache()
movie_df.show(3)
movie_df.count()

+-------+--------------------+--------------------+
|MovieID|               Title|              Genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
+-------+--------------------+--------------------+
only showing top 3 rows



10684

In [69]:
rating_schema = StructType([
    StructField("UserID", IntegerType(), True),
    StructField("MovieID", IntegerType(), True),
    StructField("Rating", FloatType(), True),
    StructField("Timestamp", IntegerType(), True)
])

rating_df = spark.read.csv(path + "ratings.dat", sep="::", header=False, schema=rating_schema, mode='DROPMALFORMED')
rating_df = rating_df.dropna()
rating_df = rating_df.filter((rating_df.Rating >= 0) & (rating_df.Rating <= 5))
rating_df = rating_df.withColumn("Date", to_date(from_unixtime("Timestamp", "yyyy-MM-dd HH:mm:ss")))
rating_df.cache()
rating_df.show(3)
rating_df.count()

+------+-------+------+---------+----------+
|UserID|MovieID|Rating|Timestamp|      Date|
+------+-------+------+---------+----------+
|     1|    122|   5.0|838985046|1996-08-02|
|     1|    185|   5.0|838983525|1996-08-02|
|     1|    231|   5.0|838983392|1996-08-02|
+------+-------+------+---------+----------+
only showing top 3 rows



10000032

**(Q1) What are the titles of top 5 most popular movies i.e. have the most ranking in the whole dataset?**

In [78]:
movie_rating_count = rating_df.groupBy("MovieID").agg(
    count("*").alias("#Ratings")).orderBy("#Ratings", ascending=False)
movie_rating_count = movie_rating_count.withColumn("Rank", monotonically_increasing_id())
popular_movies = movie_rating_count.join(movie_df, "MovieID").select(
    "Rank", "#Ratings", "Title")
popular_movies.cache()

DataFrame[Rank: bigint, #Ratings: bigint, Title: string]

In [79]:
top_n_movie = 5
popular_movies.show(top_n_movie)

+----+--------+--------------------+
|Rank|#Ratings|               Title|
+----+--------+--------------------+
|   0|   34864| Pulp Fiction (1994)|
|   1|   34457| Forrest Gump (1994)|
|   2|   33668|Silence of the La...|
|   3|   32631|Jurassic Park (1993)|
|   4|   31126|Shawshank Redempt...|
+----+--------+--------------------+
only showing top 5 rows



**(Q2) What are the top 5 ranked movie genres on average in the whole dataset?**

In [81]:
movie_avg_ratings = rating_df.groupBy("MovieID").agg(avg("Rating").alias('Avg_Rating'))
movie_avg_ratings.show(10)

+-------+------------------+
|MovieID|        Avg_Rating|
+-------+------------------+
|   1580| 3.563920531231442|
|   5300|3.7041884816753927|
|    471| 3.659111243662392|
|   1591| 2.591865858009276|
|   3175|3.6245300142616363|
|   3997| 2.072898032200358|
|   1959|3.6309438040345823|
|   2366|3.6127175743964064|
|   2866| 3.607728337236534|
|   1088|3.1912112010796223|
+-------+------------------+
only showing top 10 rows



In [83]:
movie_genres = movie_df.withColumn("Genre", explode(
    split("Genres", "\|"))).select("MovieID", "Genre")
movie_genres.show(10)
movie_genres.count()

+-------+---------+
|MovieID|    Genre|
+-------+---------+
|      1|Adventure|
|      1|Animation|
|      1| Children|
|      1|   Comedy|
|      1|  Fantasy|
|      2|Adventure|
|      2| Children|
|      2|  Fantasy|
|      3|   Comedy|
|      3|  Romance|
+-------+---------+
only showing top 10 rows



21580

In [84]:
movie_genres_rating = movie_genres.join(movie_avg_ratings,"MovieID")
movie_genres_rating.show(10)
movie_genres_rating.count()

+-------+---------+------------------+
|MovieID|    Genre|        Avg_Rating|
+-------+---------+------------------+
|   1580|   Sci-Fi| 3.563920531231442|
|   1580|   Comedy| 3.563920531231442|
|   1580|   Action| 3.563920531231442|
|   5300|  Western|3.7041884816753927|
|   5300| Thriller|3.7041884816753927|
|   5300|    Drama|3.7041884816753927|
|   5300|Adventure|3.7041884816753927|
|   5300|   Action|3.7041884816753927|
|    471|  Romance| 3.659111243662392|
|    471|  Fantasy| 3.659111243662392|
+-------+---------+------------------+
only showing top 10 rows



21573

In [88]:
genre_ratings = movie_genres_rating.groupBy("Genre").agg(
    avg("Avg_Rating").alias('Avg_Rating'))
genre_ratings = genre_ratings.orderBy('Avg_Rating', ascending = False)
genre_ratings.cache()
genre_ratings.count()

20

In [87]:
top_n_genre = 5
genre_ratings.show(top_n_genre)

+------------------+------------------+
|             Genre|        Avg_Rating|
+------------------+------------------+
|         Film-Noir|3.7118715983793593|
|(no genres listed)| 3.642857142857143|
|       Documentary|3.4621763397993477|
|               War| 3.454612791239219|
|             Drama|3.3498928844848557|
+------------------+------------------+
only showing top 5 rows



**(Q3) How many movies have been ranked the most consecutive days?**

In [60]:
movie_rating_date = rating_df.select(["MovieID","date"]).drop_duplicates()
movie_rating_date.show(10)
movie_rating_date.count()

+-------+----------+
|MovieID|      date|
+-------+----------+
|    589|1996-08-02|
|   1212|2003-04-11|
|    208|2005-05-20|
|    288|2005-03-23|
|   1080|2005-05-12|
|   1396|2005-03-24|
|   2948|2005-05-12|
|   2997|2005-03-24|
|   3452|2005-03-24|
|   5810|2005-05-12|
+-------+----------+
only showing top 10 rows



4574509

In [62]:
winspec = Window.partitionBy("MovieID").orderBy("date")
movie_rating_date = movie_rating_date.withColumn("date_diff",
    datediff("date", lag("date", 1).over(winspec)))
movie_rating_date = movie_rating_date.withColumn("winID",
    sum(when(col("date_diff") != 1, 1).otherwise(0)).over(
        winspec.rowsBetween(Window.unboundedPreceding, 0)))
movie_rating_date.show(10)

+-------+----------+---------+-----+
|MovieID|      date|date_diff|winID|
+-------+----------+---------+-----+
|      1|1996-01-29|     NULL|    0|
|      1|1996-02-01|        3|    1|
|      1|1996-02-02|        1|    1|
|      1|1996-02-05|        3|    2|
|      1|1996-02-12|        7|    3|
|      1|1996-02-22|       10|    4|
|      1|1996-02-23|        1|    4|
|      1|1996-02-26|        3|    5|
|      1|1996-03-04|        7|    6|
|      1|1996-03-05|        1|    6|
+-------+----------+---------+-----+
only showing top 10 rows



In [67]:
movie_rated_days = movie_rating_date.groupBy("MovieID", "winID").count()
movie_rated_days = movie_rated_days.groupBy("MovieID").agg(max("count").alias("max_consecutive_days"))
movie_rated_days = movie_rated_days.orderBy("max_consecutive_days", ascending=False)
movie_rated_days.cache()
movie_rated_days.show(10)

+-------+--------------------+
|MovieID|max_consecutive_days|
+-------+--------------------+
|   5952|                 622|
|   2858|                 485|
|   4993|                 483|
|    356|                 425|
|   6377|                 425|
|   2571|                 364|
|   2762|                 362|
|   1270|                 327|
|    593|                 317|
|   3578|                 307|
+-------+--------------------+
only showing top 10 rows

